In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

In [12]:
# parameters
customer_id = '134'
formatted_attribute = 'belt_width_inches'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Belt Pulley"""

regex_pattern = r"(?i)([^\.]\d* in\w* Belt Width)|(\d*\.\d* in\w* Belt Width)|(\d*\-\d*\/\d* in\w* Belt Width)|(\d*\.\d in\w* Belt Width)"
# regex_pattern = r"(?i)([^\.]\d* in Belt Width)|(\d*\/\d* in Belt Width)|(\d*\.\d* in Belt Width)|(\d* in Belt Width)"


value='%n/a%'
customer_name='%motionindustriesus%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [13]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [14]:
df = query_from_file(file_name='top_width.sql', params=params)
df

attribute         value      customers_name  external_id  \
0      belt_width_inches  0 - 1 Inches  motionindustriesus  ="00228424"   
1      belt_width_inches  0 - 1 Inches  motionindustriesus  ="00228428"   
2      belt_width_inches  0 - 1 Inches  motionindustriesus  ="00228430"   
3      belt_width_inches  0 - 1 Inches  motionindustriesus  ="00228431"   
4      belt_width_inches  0 - 1 Inches  motionindustriesus  ="00228432"   
...                  ...           ...                 ...          ...   
18878  belt_width_inches           n/a  motionindustriesus     00730674   
18879  belt_width_inches           n/a  motionindustriesus     00730673   
18880  belt_width_inches           n/a  motionindustriesus     00730672   
18881  belt_width_inches           n/a  motionindustriesus     00730644   
18882  belt_width_inches           n/a  motionindustriesus     00730668   

                                                                                                                                                       long_desc  \
0                                                                                                                                                           None   
1                                                                                                                                                           None   
2                                                                                                                                                           None   
3                                                                                                                                                           None   
4                                                                                                                                                           None   
...                                                                                                                                                          ...   
18878  1000322 Bushing Bore V-Belt Pulley - A Belt Section, 3 Groove, 4.4 in Pitch Dia., 4.65 in O.D., Split Taper Bushed, P1 Series Bushing, Cast Iron Material   
18879  1000280 Bushing Bore V-Belt Pulley - A Belt Section, 3 Groove, 4.2 in Pitch Dia., 4.45 in O.D., Split Taper Bushed, P1 Series Bushing, Cast Iron Material   
18880    1000249 Bushing Bore V-Belt Pulley - A Belt Section, 3 Groove, 4 in Pitch Dia., 4.25 in O.D., Split Taper Bushed, P1 Series Bushing, Cast Iron Material   
18881  1000199 Bushing Bore V-Belt Pulley - A Belt Section, 2 Groove, 3.8 in Pitch Dia., 4.05 in O.D., Split Taper Bushed, P1 Series Bushing, Cast Iron Material   
18882  1000082 Bushing Bore V-Belt Pulley - A Belt Section, 3 Groove, 3.2 in Pitch Dia., 3.45 in O.D., Split Taper Bushed, P1 Series Bushing, Cast Iron Material   

      resolution  
0       standard  
1       standard  
2       standard  
3       standard  
4       standard  
...          ...  
18878   standard  
18879   standard  
18880   standard  
18881   standard  
18882   standard  

[18883 rows x 6 columns]

In [15]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['0 - 1 Inches',
 '1.1 - 2 Inches',
 '2.1 - 3 Inches',
 '3.1 - 4 Inches',
 '4.1 - 5 Inches',
 'Greater than 5 Inches']

In [16]:
kwp=KeywordProcessor()
kwp.add_keywords_from_list(list_values)
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

0        []
1        []
2        []
3        []
4        []
         ..
18878    []
18879    []
18880    []
18881    []
18882    []
Name: long_desc, Length: 18883, dtype: object

In [17]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [20]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df

TypeError: expected string or bytes-like object

In [ ]:
df['matches'].explode().value_counts()

In [ ]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [ ]:
na_to_gsheet(df)

In [ ]:
matches = df[df['matches'].astype(str) != '[]'] 

In [ ]:
def remove_txt(lst):
    assert type(lst) == list, "make sure it's a list"
    tmp = []
    for measure in lst:
        val = re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.','',measure)
        if val != '':
            tmp.append(val.replace(",",''))
    if str(tmp) == '[]':
        return ''
    else:
        return '{:,}'.format(int(max(tmp)))

In [ ]:
matches['matches'] = matches['matches'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.','',str(measure)))

In [ ]:
matches['matches'].unique()

In [ ]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [ ]:
def remove_fraction(numbers):
    if numbers.find('/') > -1:
        numbers = literal_eval(numbers)
#         inte, fract = numbers.split('-')
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
#         num = int(inte) + tmp
        return str(tmp)
    else:
        return numbers

In [ ]:
matches['matches'] = matches['matches'].apply(lambda x: remove_fractions(x))

In [ ]:
matches['matches'].unique()

In [ ]:
matches['matches'] = matches['matches'].apply(lambda x: remove_fraction(x))

In [ ]:
matches['matches'].unique()

In [ ]:
matches['matches'].value_counts()

In [ ]:
matches['matches'] = matches['matches'].apply(lambda x: float(literal_eval(x)))
matches[curation_col]=matches['matches']
matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload_Pulley.csv',index=False)
    matches.to_csv(f'{drive_path}/Motion - {attribute} - match upload_Shawn.csv',index=False)

In [ ]:
looks_good('Motion Industries', attribute, df, matches)